In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/classification/accoustics/simplified')
data = pd.read_excel('cg_play_means.xlsx', engine='openpyxl', index_col=0)

In [ ]:
X = data.iloc[:,1:17].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data.iloc[:,-1]])
groups = data.index

In [ ]:
X

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups , random_state=42)
scalar = preprocessing.StandardScaler()
#clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
parameter_candidates = {'model__kernel':['linear', 'rbf'], 'model__C':[1, 10]}
clf = svm.SVC()
pipeline = Pipeline(steps=[('preprocess', scalar), ('model', clf)])

kf = GroupKFold(n_splits=5)
#cross_val_score(pipeline, X, y, cv = cv)
search = GridSearchCV(pipeline, parameter_candidates, scoring = 'accuracy',refit=True, cv =kf)
search.fit(X_shuffled, y_shuffled, groups=np.array(groups_shuffled))

In [ ]:
pd.DataFrame.from_dict(search.cv_results_)

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups , random_state=42)
scalar = preprocessing.StandardScaler()
clf = LogisticRegression()
#clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
parameter_candidates = {'model__penalty' : ['l1', 'l2'],'model__solver' : ['liblinear']
                        ,'model__C':np.logspace(-4, 4, 20)}

pipeline = Pipeline(steps=[('preprocess', scalar), ('model', clf)])

kf = GroupKFold(n_splits=5)
#cross_val_score(pipeline, X, y, cv = cv)
search = GridSearchCV(pipeline, parameter_candidates, scoring = 'accuracy',refit=True, cv =kf)
search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

In [ ]:
pd.DataFrame.from_dict(search.cv_results_)

In [ ]:
print('Best score for data:', search.best_score_) 

In [ ]:
kf = GroupKFold(n_splits=5)
accuracies= []
predictions = []
indexes =[]
ys = []

for train_index, test_index in kf.split(X,y, groups):
            # scale training and test data based on statistics of only training data
    scaler = StandardScaler()
    X_train_, X_test_ = X[train_index], X[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
   # print("Train :", train_index, "Test :", test_index)
    scaler = StandardScaler()
    scaler.fit(X_train_)
        # normalize data
    X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
    model = LogisticRegressionCV().fit(X, y)
    model.fit(X_train_, y_train_)
    #print(model.predict(X_test_))
    predictions.append(model.predict(X_test_))
    print(model.scores_)
    print(groups[test_index])
    indexes.append(groups[test_index])
    ys.append(y_test_)


In [ ]:
accuracies

In [ ]:
df = pd.DataFrame(list(zip([val for sublist in predictions for val in sublist], [val for sublist in indexes for val in sublist],
                          [val for sublist in ys for val in sublist])), columns = ['prediction', 's_id', 'lbl'])

In [ ]:
df

In [ ]:
df.groupby(['s_id']).agg(lambda x:x.value_counts().index[0])